In [8]:
import bs4 as bs
from bs4 import BeautifulSoup
import requests
import matplotlib as mpl
from matplotlib import pyplot as plt

import pandas as pd
import xlsxwriter

url = 'http://coronavirus-tracking.ru'
cor = requests.get(url)
print(cor.status_code)



numbers = []  #список численности заболеваемости
regions = []  # список регионов  рф с тегами ссылками
regions2 = [] #список без ссылок


corvir = bs.BeautifulSoup(cor.text)   #передает атрибут text ответа функции bs4.BeautifulSoup
# возвращаемый этой функцией объект BeautifulSoup сохраняется в corvir


soup = BeautifulSoup(cor.text, "html.parser")    #- код из статьи


regions = soup.findAll('a', class_='table-link')
print('длина списка регионов на которые еcть ссылки - ', len(regions))
print(type(regions))

tdElems = corvir.select('td') 
print('длина списка красных цифр заболеваемости td - ', len(tdElems))


numbers = soup.findAll('td', style = "color:red")
print ('Длина списка цифр заболеваемости - ', len(numbers))

print("Первый элемент списка чисел заболеваемости -", numbers[0].getText())

reg_12 = [] #список для первых 12 - ти регионов, так как они с ссылками

for i in range(12):
    reg_12.append(regions[i].getText())
    #print(regions[i].getText())

print('Список первых 12- ти регионов, которые с ссылками',reg_12)

num_12 = [] #список первых 12 ти цифр

for i in range(12):
    num_12.append(numbers[i].getText())

print('Список показателей заболеваемости первых 12 ти ', num_12)

for i, elem in enumerate(num_12):    #изменяем список цифр
    num_12[i] = int(elem)

print('Список показателей заболеваемости первых 12 ти . НОВЫЙ - ЦИФРЫ!', num_12)


###########ВИЗУАЛИЗАЦИЯ##############################

index = reg_12
data  = num_12


# Create a Pandas dataframe from the data.

df = pd.DataFrame(
    {"Заболеваемость" : num_12, },
    
    index = reg_12)




# Create a Pandas Excel writer using XlsxWriter as the engine.
sheet_name = 'Sheet1'
writer     = pd.ExcelWriter('pandas_chart_columns.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name=sheet_name)





# Access the XlsxWriter workbook and worksheet objects from the dataframe.
workbook  = writer.book
worksheet = writer.sheets[sheet_name]




# Create a chart object.
chart = workbook.add_chart({'type': 'bar', 'stacked': True})

# Some alternative colors for the chart.
colors = ['#E41A1C', '#377EB8']

#Configure the series of the chart from the dataframe data.
for col_num in data:
    chart.add_series(
        
        
        
        {
        'name':       ['Sheet1', 0, col_num],
          'categories': ['Sheet1', 1, 12, 12, 0],   #подписи снизу, опционально
            #1 цифра с какой позиции поставить значение заболеваемости
            #2 цифра по какую нумерация
            #3 и 4 цифра для подписей словами оси х данными из index 
                                 
          'values':     '=Sheet1!$B$2:$B$13',
         'fill':       {'color':  'green'},
          'overlap':    -10
    }
  )


    

# Configure the chart axes.

chart.set_x_axis({'name': 'Регионы', 'interval_unit': 5})
chart.set_x2_axis({'name': 'Регионы', 'interval_unit': 5 })

chart.set_y_axis({'name': 'Заболеваемость', 'major_gridlines': {'visible': False}})

chart.set_size({'width': 720, 'height': 576})
chart.set_title({'name' :'Заболеваемость Covid-19'})



# Insert the chart into the worksheet.
worksheet.insert_chart('G2', chart)

# Close the Pandas Excel writer and output the Excel file.
writer.save()





200
длина списка регионов на которые еcть ссылки -  49
<class 'bs4.element.ResultSet'>
длина списка красных цифр заболеваемости td -  1100
Длина списка цифр заболеваемости -  273
Первый элемент списка чисел заболеваемости - 3357
Список первых 12- ти регионов, которые с ссылками ['Москва', 'Московская область', 'Санкт-Петербург', 'Республика Коми', 'Ленинградская область', 'Свердловская область', 'Краснодарский край', 'Республика Бурятия', 'Пензенская область', 'Челябинская область', 'Красноярский край', 'Нижегородская область']
Список показателей заболеваемости первых 12 ти  ['3357', '260', '171', '59', '47', '40', '39', '35', '27', '26', '25', '25']
Список показателей заболеваемости первых 12 ти . НОВЫЙ - ЦИФРЫ! [3357, 260, 171, 59, 47, 40, 39, 35, 27, 26, 25, 25]
